**Inclass material for Week 4: SQL Query using `pandas`**

This notebook was made based on main materials `4_SQL_Query.ipynb`

Version: Newton - March 2021

---
**START OF DAY 1**

# SQL Query using `pandas`

**Training Objectives**

- Querying from SQL Databases
- SQL Joins
- SQL Conditional Statements
- Flavors and Common Operators
- End to end data analysis

# Working with SQL Databases

## Database Schema

Database `chinook.db` terdiri dari 11 tabel yang berisi sampel data dari sebuah toko yang menjual media digital:

- Data terkait media seperti pada tabel `tracks`, `albums`, `artists`, `genres`, `media_types`, `playlists`, `playlist_track` merupakan data asli dari library Apple iTunes.
- Informasi mengenai `customers` dan `employees` dibuat menggunakan nama dan alamat fiktif yang dapat ditemukan di Google maps, dan data lain yang diformat dengan baik (telepon, fax, email, dll).
- Informasi penjualan `invoices` dan `invoice_items` dihasilkan secara otomatis menggunakan data acak untuk periode tahun 2009-2013.

Skema berikut sering disebut sebagai **Entity Relationship Diagram (ERD)**, menunjukkan:
1. Entitas (Tabel)
2. Atribut (Kolom beserta tipe datanya)
3. Kardinalitas (Hubungan antar tabel)

![](assets/chinookschema2.png)

## Database Connection

Terdapat banyak package Python yang menyediakan fungsionalitas agar data analyst dapat bekerja dengan basis data (database). Salah satu contohnya, apabila kita ingin bekerja dengan database MySQL, maka kita dapat menggunakan `pymysql` seperti berikut:

```
import pymysql
conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    db=database)
```

Kemudian untuk membaca data kita menggunakan `pd.read_sql_query()` dan menyertakan connection yang telah dibuat:

```
sales = pd.read_sql_query("SELECT * FROM sales", conn)
```

Dibalik layar, `pandas` menggunakan menggunakan [SQLAlchemy](https://www.sqlalchemy.org/) sehingga setiap database dapat bekerja. Tenang saja, hal ini bukan sesuatu yang perlu Anda khawatirkan pada pembelajaran ini. Sebagai tahap awal, mari kita coba bagaimana mengkoneksikan Jupyter Notebook dengan database SQLite (menggunakan package `sqlite3`) yang disebut sebagai **connection**:

In [1]:
import sqlite3 # default package: Database API (DBAPI)
import pandas as pd

In [2]:
# membuat objek connection
conn = sqlite3.connect("data_input/chinook.db")
conn

## `SELECT` Statements

`SELECT` digunakan untuk memilih kolom dari sebuah tabel.

Syntax `SELECT`:

```
SELECT <NAMA_KOLOM>
FROM <NAMA_TABLE>
```

Query di bawah menunjukkan bahwa kita ingin mengambil **semua kolom** (ditandai oleh command `SELECT *`) pada tabel `albums`.

In [3]:
# query: mengambil semua kolom pada tabel albums
# con: nama parameter
# conn: object connection yang kita buat sebelumnya
pd.read_sql_query("SELECT * FROM albums", con=conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [4]:
# misal saya mau ambil kolom AlbumId dan Title saja dari tabel albums
pd.read_sql_query("SELECT AlbumId, Title FROM albums", con=conn)

,AlbumId,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,3,Restless and Wild
3,4,Let There Be Rock
4,5,Big Ones
...,...,...
342,343,Respighi:Pines of Rome
343,344,Schubert: The Late String Quartets & String Qu...
344,345,Monteverdi: L'Orfeo
345,346,Mozart: Chamber Music


💡 **Note**: Berbeda dari Python, perintah yang ada di SQL bersifat case **insensitive**. Query di atas dapat juga dituliskan sebagai `select * from albums`. Namun, demi kemudahan pembacaan, statement biasa dituliskan dengan huruf besar (dalam hal ini `SELECT` dan `FROM`).

In [5]:
# SQL: case insensitive
# besar kecil bisa, namun TIDAK direkomendasikan
pd.read_sql_query("sEleCt * fRoM aLbuMs", con=conn)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


### Knowledge Check

We'll create a `DataFrame`: this time select all columns from the `artists` table. Recall that when we use `pd.read_sql_query()` command we pass in the SQL query as a string, and add a connection as the second parameter. Save the output as a `DataFrame`.

Your DataFrame should be constructed like this:

`__ = pd.read_sql_query("SELECT __ FROM __ ", conn)`

**Question:** How many rows are there in your DataFrame?

In [6]:
# your code here
# Pak Calvin
artists = pd.read_sql_query("SELECT * FROM artists", conn)
artists.shape[0]

275

Kita akan sering menggunakan parameter berikut dalam method `pd.read_sql_query()`:

- `sql`: SQL query dalam bentuk string
- `con`: SQL connection
- `index_col`: nama atau index kolom yang ingin dijadikan index (seperti pada `pd.read_csv()`)
- `parse_dates`: nama kolom yang ingin dikonversi menjadi tipe data `datetime64` (seperti pada `pd.read_csv()`)

Silahkan kunjungi [official documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) untuk detail lebih lanjut.

### LIMIT

Statement `LIMIT` digunakan untuk mengambil beberapa baris pertama pada data.

Contoh: Ambil **5 baris pertama** dari tabel `artists` lalu jadikan kolom `ArtistId` sebuah index dengan parameter `index_col`.

In [7]:
pd.read_sql_query("SELECT * FROM artists LIMIT 5", conn)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [8]:
# statement OFFSET: skip berapa baris pertama
pd.read_sql_query("SELECT * FROM artists LIMIT 5 OFFSET 270", conn)

,ArtistId,Name
0,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
1,272,Emerson String Quartet
2,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3,274,Nash Ensemble
4,275,Philip Glass Ensemble


In [9]:
# Bu Felicia: fetching data untuk keseluruhan baris lalu subsetting dengan .tail()
pd.read_sql_query('SELECT * FROM artists',con=conn).tail()

,ArtistId,Name
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble
274,275,Philip Glass Ensemble


## SQL Joins

Statement `JOIN` digunakan untuk **menggabungkan dua tabel** menjadi satu tabel melalui **kolom penghubung** yang sama. Operasi `JOIN` dapat dilakukan lebih dari satu kali dalam satu SQL query. Berikut adalah diagram ilustrasi beberapa macam operasi SQL `JOIN`:

![](assets/sqljoins.png)
Credit: Data & Object Factory, LLC

- `LEFT JOIN` paling sering digunakan pada sebagian besar skenario bisnis. `LEFT JOIN` mengembalikan semua baris pada tabel kiri **terlepas** dari apakah ada baris yang cocok pada tabel kanan.
- `INNER JOIN` adalah tipe join yang sangat intuitif dan mudah dipahami. Query ini mengembalikan semua baris di tabel kiri yang cocok dengan tabel kanan.

**Note**:

- `RIGHT JOIN` hampir tidak pernah digunakan karena ekuivalen dengan `LEFT JOIN` hanya mengganti peletakkan tabel kiri dan kanannya saja.
- `FULL OUTER JOIN` sangat jarang digunakan. Selain alasan komputasi, tipe join ini mengembalikan semua baris dari kedua tabel terlepas dari apakah ada kecocokan atau tidak, sehingga menghasilkan DataFrame dengan banyak missing value.

Ilustrasi terkait ke-empat tipe join dapat dilihat pada [Visual JOIN](https://joins.spathon.com/).

Syntax `JOIN`:

```
SELECT <COLUMNS>
FROM <LEFT_TABLE>
[LEFT|INNER] JOIN <RIGHT_TABLE>
ON <LEFT_TABLE>.key = <RIGHT_TABLE>.key
```

Perhatikan kembali skema database `chinook.db`:

1. Tabel `albums`: 
    - `AlbumId`
    - `Title`
    - `ArtistId`


2. Tabel `artists`:
    - `ArtistId`
    - `Name` 
    
**Kasus:** Kita ingin mendapatkan DataFrame yang menampilkan semua baris dengan kolom `AlbumId`, `Title`, dan `Name`. Perhatikan bahwa kolom `Name` terdapat pada tabel `artists`, sedangkan kolom `AlbumId` dan `Title` terdapat pada tabel `albums`.

Kira-kira bagaimana strategi yang tepat? Solusi yang paling mudah adalah menggunakan `LEFT JOIN`. Mari kita lihat contohnya:

In [10]:
albums = pd.read_sql_query("SELECT AlbumId, Title, Name \
                            FROM albums \
                            LEFT JOIN artists \
                            ON artists.ArtistId = albums.ArtistId", conn)
albums.head()

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith


Perhatikan SQL query di atas:
- Tabel `albums` menjadi tabel kiri, sedangkan tabel `artists` menjadi tabel kanan.
- Kedua tabel digabungkan menggunakan operasi `LEFT JOIN` melalui key `ArtistId`.
- Penggunaan karakter backslash (`\`) ditujukan agar ke-empat baris SQL query dibaca sebagai satu kesatuan baris. Namun penggunaan backslash kurang efisien dan memiliki resiko error yang lebih tinggi. Sebagai alternatif, Anda dapat mengapit SQL query dengan `'''` (petik tiga kali)

In [11]:
# re-create tabel di atas menggunakan petik tiga kali '''
pd.read_sql_query(
    '''
    SELECT AlbumId, Title, Name
    FROM albums
    LEFT JOIN artists
    ON albums.ArtistId = artists.ArtistId
    ''', conn
)

# ekuivalen:
# albums.ArtistId = artists.ArtistId
# artists.ArtistId = albums.ArtistId

,AlbumId,Title,Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith
...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


Terkadang kita memiliki nama kolom ataupun tabel yang panjang dan redundan untuk diketik.

Penggunaan statement `AS` dapat digunakan untuk melakukan **aliasing** nama tabel dan nama kolom.

In [12]:
# re-create tabel di atas menggunakan aliasing
pd.read_sql_query(
    '''
    SELECT AlbumId, Title AS AlbumTitle, Name AS 'Artist Name'
    FROM albums AS alb
    LEFT JOIN artists AS art
    ON alb.ArtistId = art.ArtistId
    ''', conn
)

,AlbumId,AlbumTitle,Artist Name
0,1,For Those About To Rock We Salute You,AC/DC
1,2,Balls to the Wall,Accept
2,3,Restless and Wild,Accept
3,4,Let There Be Rock,AC/DC
4,5,Big Ones,Aerosmith
...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,Nash Ensemble


⚠️ Hati-hati! Saat melakukan `LEFT JOIN`, peletakkan tabel kiri dan kanan sangat berpengaruh. Sebagai perbanding, sekarang kita tukar tabel `artists` menjadi tabel kiri, sedangkan `albums` menjadi tabel kanan.

Tabel `x` di bawah akan menampilkan semua data `artists` walaupun artist tersebut tidak ada di tabel `albums`.

In [13]:
x = pd.read_sql_query(
    '''
    SELECT AlbumId, Title, Name
    FROM artists
    LEFT JOIN albums
    ON artists.ArtistId = albums.ArtistId
    ''', conn)

# cek baris dengan missing value
x[x.isna().any(axis=1)]

,AlbumId,Title,Name
50,NaN,None,Milton Nascimento & Bebeto
51,NaN,None,Azymuth
55,NaN,None,João Gilberto
56,NaN,None,Bebel Gilberto
57,NaN,None,Jorge Vercilo
...,...,...,...
327,NaN,None,DJ Dolores & Orchestra Santa Massa
328,NaN,None,Seu Jorge
329,NaN,None,Sabotage E Instituto
330,NaN,None,Stereo Maracana


### Knowledge Check

Consider the database schema illustration again and pay attention to the following tables and their respective columns:

1. `tracks`: `TrackId`, `Name`, `AlbumId`, `GenreId`, ... `UnitPrice` 
    
2. `albums`: `AlbumId`, `Title`, `ArtistId`

3. `artists`: `ArtistId`, `Name`

4. `genres`: `GenreId`, `Name`

**Instruction**: Create a `DataFrame` containing all columns from the `tracks` table; Additionally, it should also contain:

- The `Title` column from the `albums` table
- The `Name` column from the `artists` table
- The `Name` column from the `genres` table

> **Hint 1**: In your `SELECT` statement, you can use `SELECT tracks.* FROM TRACKS` to select all columns from the `TRACKS` table
> 
> **Hint 2**: When we write `SELECT tracks.Name AS tracksName`, we are renaming the output column from `Name` to `tracksName` using a technique called column aliasing. You may optionally consider doing this for columns that share the same name across different tables 

- Set the `TrackId` column to be the index. -> parameter `index_col`
- Give your `DataFrame` a name: name it `tracks`.
- Verify: the resulting `DataFrame` should has 3503 rows and 11 columns. 

In [14]:
# your code here
# Pak Sayyid

# Syntax: nama_table.nama_kolom
tracks = pd.read_sql_query(
    '''
    SELECT 
    tracks.*,
    albums.Title, 
    artists.Name AS "Artist Name",
    genres.Name AS "Genre Name"
    FROM tracks
    LEFT JOIN albums
    ON tracks.AlbumId=albums.AlbumId
    LEFT JOIN artists
    ON albums.ArtistId=artists.ArtistId
    LEFT JOIN genres
    ON tracks.GenreId=genres.GenreId
    ''', conn, index_col= 'TrackId'
)
# tracks.shape
tracks

,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,Artist Name,Genre Name
TrackId,,,,,,,,,,,
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,AC/DC,Rock
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,Accept,Rock
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Restless and Wild,Accept,Rock
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,Accept,Rock
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Restless and Wild,Accept,Rock
...,...,...,...,...,...,...,...,...,...,...,...
3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Respighi:Pines of Rome,Eugene Ormandy,Classical
3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Classical
3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Classical


**END OF DAY 1**

---

**START OF DAY 2**

Perform EDA on `tracks` to answer the following question:

1. Use `tail()` to inspect the last 5 rows of data. Which genre is present in the last 5 rows of our `tracks` DataFrame (Check all that apply)?
    - [ ] Latin
    - [x] Classical
    - [x] Soundtrack
    - [ ] Pop

In [15]:
# your code here

# Bu Ani
tracks["Genre Name"].tail()

TrackId
3499     Classical
3500     Classical
3501     Classical
3502     Classical
3503    Soundtrack
Name: Genre Name, dtype: object

2. Apply `pd.crosstab(..., columns='count')`, `.value_counts()`, or any other techniques you've learned to compute the frequency table of Genres in your DataFrame. Which is among the top 3 most represented genres in the `tracks` DataFrame?
    - [x] Latin
    - [ ] Classical
    - [ ] Soundtrack
    - [ ] Pop

In [16]:
# your code here

# Pak Aldo
tracks['Genre Name'].value_counts().head(3)

Rock     1297
Latin     579
Metal     374
Name: Genre Name, dtype: int64

3. Use `groupby()` on Artist Name and compute the `mean()` on the `UnitPrice` of each tracks. You will realize that most artists price their tracks at 0.99 (`mean`) but there are several artists where the `mean()` is 1.99. Which of the Artist has a mean of 0.99 `UnitPrice`:
    - [ ] The Office
    - [ ] Aquaman
    - [x] Pearl Jam
    - [ ] Lost

In [17]:
# Pak Calvin
artist_mean_price = tracks.groupby('Artist Name').mean()[['UnitPrice']]
artist_mean_price.sort_values('UnitPrice', ascending=False).head(10)

,UnitPrice
Artist Name,
Lost,1.99
Heroes,1.99
Battlestar Galactica (Classic),1.99
Aquaman,1.99
Battlestar Galactica,1.99
The Office,1.99
Iron Maiden,0.99
Lenny Kravitz,0.99
Pearl Jam,0.99


In [18]:
artist_mean_price.loc[['The Office', 'Aquaman', 'Pearl Jam', 'Lost']]

,UnitPrice
Artist Name,
The Office,1.99
Aquaman,1.99
Pearl Jam,0.99
Lost,1.99


In [19]:
# your code here
pd.options.display.float_format = '{:.20f}'.format
artist_mean_price

,UnitPrice
Artist Name,
AC/DC,0.98999999999999999112
Aaron Copland & London Symphony Orchestra,0.98999999999999999112
Aaron Goldberg,0.98999999999999999112
Academy of St. Martin in the Fields & Sir Neville Marriner,0.98999999999999999112
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,0.98999999999999999112
...,...
Vinícius De Moraes,0.99000000000000010214
Wilhelm Kempff,0.98999999999999999112
Yehudi Menuhin,0.98999999999999999112


In [20]:
# conditional subsetting: setelah pembulatan 2 angka belakang koma
artist_mean_price[artist_mean_price['UnitPrice'] == 1.99]

,UnitPrice
Artist Name,
Aquaman,1.98999999999999999112


In [21]:
# conditional subsetting: setelah pembulatan 2 angka belakang koma
artist_mean_price[artist_mean_price['UnitPrice'].round(2) == 1.99]

,UnitPrice
Artist Name,
Aquaman,1.98999999999999999112
Battlestar Galactica,1.98999999999999976907
Battlestar Galactica (Classic),1.99000000000000021316
Heroes,1.99000000000000021316
Lost,1.99000000000000110134
The Office,1.98999999999999843681


In [22]:
pd.reset_option('display.float_format')
artist_mean_price

,UnitPrice
Artist Name,
AC/DC,0.99
Aaron Copland & London Symphony Orchestra,0.99
Aaron Goldberg,0.99
Academy of St. Martin in the Fields & Sir Neville Marriner,0.99
Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner,0.99
...,...
Vinícius De Moraes,0.99
Wilhelm Kempff,0.99
Yehudi Menuhin,0.99


## SQL Aggregation

Di `pandas`, kita bisa menggunakan ketiga method berikut untuk membuat table agregasi:

- `.crosstab()`
- `.pivot_table()`
- `.groupby()`

Sedangkan di SQL, kita menggunakan statement `GROUP BY` yang diletakkan setelah `SELECT ... FROM ...`.

Misal kita ingin mengetahui top 5 `CustomerId` berdasarkan jumlah besaran transaksinya (`Total`) dan juga tampilkan banyak transaksinya:

In [23]:
top_cust = pd.read_sql_query("SELECT CustomerId, SUM(Total) AS TotalValue, \
                              COUNT(InvoiceId) AS Purchases \
                              FROM invoices \
                              GROUP BY CustomerId \
                              ORDER BY TotalValue DESC \
                              LIMIT 5", conn, index_col='CustomerId')
top_cust

,TotalValue,Purchases
CustomerId,,
6,49.62,7
26,47.62,7
57,46.62,7
45,45.62,7
46,45.62,7


In [24]:
# re-create the table above

# agregation: GROUP BY, lalu menerapkan fungsi aggregasi di SELECT
# mengurutkan: ORDER BY
pd.read_sql_query(
    '''
    SELECT CustomerId, SUM(Total) AS TotalValue, COUNT(InvoiceId) AS Purchases
    FROM invoices
    GROUP BY CustomerId
    ORDER BY TotalValue DESC
    LIMIT 5
    ''', conn
)

,CustomerId,TotalValue,Purchases
0,6,49.62,7
1,26,47.62,7
2,57,46.62,7
3,45,45.62,7
4,46,45.62,7


Perhatikan bagaimana query di atas dapat mengambil 5 customer teratas dari tabel `invoices`:

1. Data dikelompokkan berdasarkan `CustomerId` dengan statement `GROUP BY`
2. Terdapat dua fungsi agregasi yang digunakan: `SUM()` dan `COUNT()`, masing-masing mengagregasikan kolom `Total` dan `InvoiceId`. Perhatikan bahwa kolom yang diagregasi harus berupa data numerik dengan aggregate function: `SUM`,` AVG`, `COUNT`, ` MIN`, dan `MAX`.
3. Statement `ORDER BY` ditambahkan untuk mengurutkan tabel berdasarkan kolom `TotalValue` secara `DESC`.

💡 **Note**:

- SQL Statement `GROUP BY` ekuivalen dengan `.groupby()` pada `pandas`
- SQL Statement `ORDER BY` ekuivalen dengan `.sort_values()` pada `pandas`

### Knowledge Check

Edit the following code to find out the most popular `genres` from all invoice sales. Use different column to acquire the following information: Summation of `Total` sales, and number of tracks bought from the `Quantity` columns.

```
top_genre = pd.read_sql_query(
    '''
    SELECT genres.GenreId, genres.Name,
    _____(invoices.Total), _____(invoice_items.Quantity)
    FROM invoices
    LEFT JOIN _____ ON _____
    LEFT JOIN _____ ON _____
    LEFT JOIN _____ ON _____
    GROUP BY _____
    ORDER BY _____
    ''',
    conn,
    index_col='GenreId'
)
```

**Question:** What are the top 5 genres that generated the most profit?

In [25]:
# your code here

# Bu Ika
top5_genre = pd.read_sql_query(
    '''
    SELECT genres.GenreId, genres.Name AS GenreName,
    SUM(invoices.Total) AS Total, SUM(invoice_items.Quantity) AS Quantity
    FROM invoices
    LEFT JOIN invoice_items ON invoice_items.Invoiceid = invoices.Invoiceid
    LEFT JOIN tracks ON invoice_items.Trackid = tracks.Trackid
    LEFT JOIN genres ON genres.Genreid = tracks.Genreid
    GROUP BY GenreName
    ORDER BY Total DESC
    LIMIT 5
    ''',
    conn,
    index_col='GenreId'
)
top5_genre

,GenreName,Total,Quantity
GenreId,,,
1,Rock,7720.02,835
7,Latin,3472.55,386
3,Metal,2093.13,264
4,Alternative & Punk,1961.66,244
19,TV Shows,817.71,47


In [26]:
pd.read_sql_query(
    '''
    SELECT genres.GenreId, genres.Name AS GenreName,
    SUM(invoices.Total) AS Total, SUM(invoice_items.Quantity) AS Quantity
    FROM invoices
    LEFT JOIN invoice_items ON invoice_items.Invoiceid = invoices.Invoiceid
    LEFT JOIN tracks ON invoice_items.Trackid = tracks.Trackid
    LEFT JOIN genres ON genres.Genreid = tracks.Genreid
    GROUP BY genres.Name
    ORDER BY Total DESC
    LIMIT 5
    ''', conn
)

,GenreId,GenreName,Total,Quantity
0,1,Rock,7720.02,835
1,7,Latin,3472.55,386
2,3,Metal,2093.13,264
3,4,Alternative & Punk,1961.66,244
4,19,TV Shows,817.71,47


In [27]:
pd.read_sql_query(
    '''
    SELECT *
    FROM genres
    ''', conn
)

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll
5,6,Blues
6,7,Latin
7,8,Reggae
8,9,Pop
9,10,Soundtrack


💡 Panduan step-by-step:

1. Lakukan operasi `JOIN` tergantung dari kebutuhan kolom yang ingin dianalisis
2. Gunakan `GROUP BY` apabila ingin membuat tabel agregasi
3. Aplikasikan fungsi agregat di `SELECT`
4. Tambahkan kolom yang relevan untuk ditampilkan pada `SELECT`
5. Urutkan data menggunakan `ORDER BY`

## `WHERE` Statements

Sampai di sini, kita telah mempelajari beberapa statement SQL yang sering digunakan:

- `SELECT` statement
- SQL `JOIN`
- Aliasing
- SQL Aggregation dengan `GROUP BY`

Sekarang, kita akan melihat teknik untuk melakukan conditional subsetting atau filter baris menggunakan statement `WHERE` yang diikuti dengan **kondisi**.

### Logical Operator

- Kondisi `WHERE` dapat dikombinasikan dengan logical operator: `IS`, `AND`, `OR`, dan `NOT`:
    - `IS` sama saja seperti notasi matematis `=`
    - `IS NOT` seperti notasi `!=`
    
- Kondisi pada `WHERE` juga mendukung operator matematis seperti >, >=, <, dan <=

Contoh: kita ingin melakukan analisis terhadap semua data `invoices` yang terjadi di `BillingCountry` Germany, maka kita bisa menambahkan statement `WHERE` sebagai berikut:

- SQL `=` untuk membandingkan, sedangkan `pandas` `==`
- Walaupun SQL case insensitive, tapi kalau sedang berhubungan dengan sebuah nilai maka huruf kapitalnya disesuaikan

In [28]:
pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE BillingCountry = 'Germany'
    ''',
    conn)

# ekuivalen dengan BillingCountry IS 'Germany'

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
2,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
3,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
4,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
5,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
6,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
7,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
8,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
9,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91


Berlawanan dengan kasus di atas, misal kita ingin analisis semua data `invoices` dimana `BillingCountry` **selain** negara Germany:

In [29]:
pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE BillingCountry != 'Germany'
    ''',
    conn)

# ekuivalen dengan BillingCountry IS NOT 'Germany'

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
379,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
380,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
381,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
382,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


Kondisi yang digunakan juga dapat lebih dari satu. Misal sekarang mengambil data `invoices` yang terjadi di `BillingCountry` Canada dan juga USA:

In [30]:
pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE (BillingCountry IS 'Canada') OR (BillingCountry IS 'USA')
    ''', conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98
...,...,...,...,...,...,...,...,...,...
142,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
143,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
144,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
145,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


In [31]:
# menampilkan data transaksi yang terjadi di negara Canada dan di kota Edmonton
pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE (BillingCountry IS 'Canada') AND (BillingCity IS 'Edmonton')
    ''', conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,133,14,2010-08-13 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,1.98
2,156,14,2010-11-15 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,3.96
3,178,14,2011-02-17 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,5.94
4,230,14,2011-10-08 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,0.99
5,351,14,2013-03-31 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,1.98
6,362,14,2013-05-11 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,13.86


### `IN` Operator

Operator `IN` yang memungkinkan kita menentukan beberapa nilai untuk perbandingan. Misalnya, seperti pada kasus sebelumnya, kita ingin mengambil semua data `invoices` yang terjadi di `BillingCountry` Canada dan juga USA:

In [32]:
america = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE BillingCountry IN ('Canada', 'USA')
    ''',
    conn)

america.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
2,13,16,2009-02-19 00:00:00,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,0.99
3,14,17,2009-03-04 00:00:00,1 Microsoft Way,Redmond,WA,USA,98052-8300,1.98
4,15,19,2009-03-04 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,1.98


### Knowledge Check

Edit the following code to include a `WHERE` clause. We want the returned DataFrame to contain only the `Pop` genre and only when the `UnitPrice` of the track is 0.99:

```
popmusic = pd.read_sql_query(
    '''
    SELECT tracks.*, genres.Name AS GenreName
    FROM _____
    LEFT JOIN _____
    ON _____
    WHERE _____
    ''',
    conn,
    index_col='TrackId'
)
```

**Question:** How many rows are there in `popmusic`?

In [33]:
# your code here

# Pak Lazuardi
popmusic = pd.read_sql_query(
    '''
    SELECT tracks.*, genres.Name AS GenreName
    FROM tracks
    LEFT JOIN genres
    ON tracks.GenreId = genres.GenreId
    WHERE GenreName = 'Pop' AND UnitPrice = 0.99
    ''',
    conn,
    index_col='TrackId'
)
popmusic.shape

(48, 9)

In [34]:
type(popmusic.shape)

tuple

In [35]:
# shape -> attribute
# [] -> subsetting
popmusic.shape[1]

9

### Operating Dates

Pada operasi kondisi pada statement `WHERE` sebelumnya, kita dapat mengambil semua data `invoices` pada negara Germany. Namun, juga umum untuk kita melakukan conditional query untuk mengambil data pada **rentang tanggal** tertentu.

Sebelum lanjut, mari kita lihat tipe data dari objek `germany` yang telah diperoleh sebelumnya:

In [36]:
germany = pd.read_sql_query("SELECT * FROM invoices WHERE BillingCountry = 'Germany'", conn)
germany.dtypes

InvoiceId              int64
CustomerId             int64
InvoiceDate           object
BillingAddress        object
BillingCity           object
BillingState          object
BillingCountry        object
BillingPostalCode     object
Total                float64
dtype: object

Perhatikan bahwa `InvoiceDate` dikenali sebagai tipe data `object`. Method `pd.read_sql_query()` berperilaku seperti method `pd.read_csv()` dimana secara default membaca tipe data suatu kolom sebagai numerik dan objek. Ini tidak berarti bahwa kolom tersebut disimpan menggunakan format string (umumnya dikenal sebagai `VARCHAR` dalam database SQL). Lihatlah skema tabel berikut:

In [37]:
invoices_table = pd.read_sql_query(
    '''
    SELECT sql
    FROM sqlite_master
    WHERE name = 'invoices'
    ''', conn)
print(invoices_table.loc[0,:].values[0])

CREATE TABLE "invoices"
(
    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY ([CustomerId]) REFERENCES "customers" ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


Perlu diingat bahwa untuk DBMS yang berbeda-beda, maka cara mengambil skema tabel akan berbeda pula. Query di atas khusus untuk mengambil skema tabel dari database SQLite (`sqlite_master`).

Output dari query di atas termasuk dalam tipe Data Definition Language (DDL) yang digunakan untuk membuat tabel. Dengan membaca DDL, akan berguna untuk memahami skema tabel dari database sehingga kita dapat melakukan operasi yang sesuai. Pada skema tabel `invoices` terdapat informasi yang berguna seperti:

- `InvoiceId` sebagai primary key
- `InvoiceDate` disimpan sebagai tipe data `DATETIME` (format `YYYY-MM-DD HH:MM:SS`)
- `CustomerId` sebagai foreign key pada tabel `customers`

Jika Anda tidak disediakan dengan skema database dalam bentuk diagram, maka luangkan waktu untuk mempelajari setiap skema tabel melalui DDL-nya. **Kasus**: Kita ingin meninjau penjualan (`invoices`) tahun lalu di seluruh negara pada tahun 2012 saja.

In [38]:
invoice_2012 = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE InvoiceDate >= '2012-01-01' AND InvoiceDate <= '2012-12-31'
    ''',
    conn,
    parse_dates='InvoiceDate'
)
 
invoice_2012['InvoiceDate'].describe()

C:\Users\tomyt\anaconda3\envs\data-analytics\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  # This is added back by InteractiveShellApp.init_path()


count                      83
unique                     71
top       2012-01-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

### `BETWEEN` Operator

Melanjutkan kasus sebelumnya, apabila kita ingin menggunakan kondisi pada rentang tertentu, pendekatan yang umum digunakan adalah operator `BETWEEN`. Silahkan lengkapi code berikut dan lihat apakah data yang terambil sama seperti sebelumnya:

```
invoice_2012 = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE _____ BETWEEN '_____' AND '_____'
    ''',
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()
```

Note, ekuivalen:

- `InvoiceDate >= '2012-01-01' AND InvoiceDate <= '2012-12-31'`
- `InvoiceDate BETWEEN '2012-01-01' AND '2012-12-31'`

In [39]:
# your code here

# Pak Calvin
invoice_2012 = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-31'
    ''',
    conn,
    parse_dates='InvoiceDate'
)

invoice_2012['InvoiceDate'].describe()

C:\Users\tomyt\anaconda3\envs\data-analytics\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


count                      83
unique                     71
top       2012-01-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

⚠️ **Hati-hati!**

Operator `BETWEEN` sejatinya adalah inclusive, di mana kondisi start dan endnya termasuk. Namun, saat kita membandingkan date (misal '2012-12-31') dengan datetime (pada kolom `InvoiceDate`), seakan-akan end tidak inclusive. Sebagai eksperimen, cobalah ganti kondisi pada statement `WHERE` menjadi:

```
WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
```

Maka `InvoiceDate` pada tanggal `2012-12-30` tidak masuk.

Sebagai solusinya, lebih baik kita menambahkan komponen waktu (time) pada kondisi:
```
WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 00:00:00'
```

In [40]:
# bukti bahwa operator BETWEEN inclusive pada start dan end
pd.read_sql_query(
    '''
    SELECT * FROM invoices
    WHERE InvoiceId BETWEEN 2 AND 5
    ''',
    conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
1,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
2,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
3,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [41]:
# masalah: ketika hanya menggunakan '2012-12-30' maka data tanggal 30 Desember 2012 TIDAK masuk
pd.read_sql_query(
    '''
    SELECT * FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01' AND '2012-12-30'
    ''',
    conn,
    parse_dates='InvoiceDate'
)['InvoiceDate'].describe()

C:\Users\tomyt\anaconda3\envs\data-analytics\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  if __name__ == '__main__':


count                      82
unique                     70
top       2012-02-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-29 00:00:00
Name: InvoiceDate, dtype: object

In [42]:
# solusi: masukkan komponen TIME (00:00:00) agar data transaksi pada tanggal 30 Desember 2012 juga termasuk
pd.read_sql_query(
    '''
    SELECT * FROM invoices
    WHERE InvoiceDate BETWEEN '2012-01-01 00:00:00' AND '2012-12-30 00:00:00'
    ''',
    conn,
    parse_dates='InvoiceDate'
)['InvoiceDate'].describe()

C:\Users\tomyt\anaconda3\envs\data-analytics\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  if __name__ == '__main__':


count                      83
unique                     71
top       2012-01-22 00:00:00
freq                        2
first     2012-01-01 00:00:00
last      2012-12-30 00:00:00
Name: InvoiceDate, dtype: object

Highlight: ketika membandingkan 2 buah nilai, pastikan tipe datanya sama

### `LIKE` Operator

Operator `LIKE` sangat berguna jika kita perlu mencocokan bagian tertentu dari sebuah string daripada menggunakan operator sama dengan (`=`). `'107%'` yang Anda lihat dalam query ditujukan untuk mengekstrak nilai `BillingPostalCode` yang **dimulai** dengan angka 107. Ini sangat membantu ketika Anda ingin mengekstrak data hanya pada wilayah tertentu. Di negara Germany, kita akan tahu bahwa Wilmersdorf dan Tempelhof di Berlin memiliki kode pos dimulai dengan 107.

**Note:** Karakter `%` disebut juga sebagai [wildcard character](https://www.w3schools.com/sql/sql_wildcards.asp)

In [43]:
pd.read_sql_query(
    '''
    SELECT * FROM invoices
    WHERE BillingCountry = 'Germany'
    AND BillingPostalCode LIKE '107%'
    ''',
    conn
)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
6,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
7,224,36,2011-09-20 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
8,225,38,2011-09-20 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
9,236,38,2011-10-31 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,13.86


**Diskusi:**

- Jika `BillingPostalCode LIKE '107%'`, maka semua baris dengan kode pos **dimulai** angka 107 akan tampil.
- Jika `BillingPostalCode LIKE '%107'`, maka semua baris dengan kode pos **diakhiri** angka 107 akan tampil.

Menurut Anda apa yang akan muncul jika kita menggunakan `%` **sebelum dan sesudah** pencocokan pola?

In [44]:
pd.read_sql_query(
    '''
    SELECT * FROM invoices
    WHERE BillingPostalCode LIKE '%107%'
    ''',
    conn
)

# %107% -> mencocokan pola bebas mau di awal, tengah, ataupun akhir (yang penting contain 107)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
1,29,36,2009-05-05 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,1.98
2,30,38,2009-05-06 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,3.96
3,40,36,2009-06-15 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,13.86
4,52,38,2009-08-08 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,5.94
5,85,45,2010-01-08 00:00:00,Erzsébet krt. 58.,Budapest,None,Hungary,H-1073,1.98
6,95,36,2010-02-13 00:00:00,Tauentzienstraße 8,Berlin,None,Germany,10789,8.91
7,96,45,2010-02-18 00:00:00,Erzsébet krt. 58.,Budapest,None,Hungary,H-1073,21.86
8,104,38,2010-03-29 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,0.99
9,151,45,2010-10-19 00:00:00,Erzsébet krt. 58.,Budapest,None,Hungary,H-1073,8.91


#### Knowledge Check

**Kasus**: Pada 5 data pertama `customerinv`, kita bisa melihat kolom `Company` mungkin tidak dapat diandalkan karena bernilai `None`. Tetapi jika Anda memperhatikan kolom `Email`, Anda dapat melihat beberapa customer memiliki domain email `apple`, yang bisa menjadi indikator perusahaan mereka.

In [45]:
customerinv = pd.read_sql_query(
    '''
    SELECT FirstName, LastName, Email, Company,
    InvoiceId, InvoiceDate, BillingCountry, Total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    ''',
    conn)

customerinv.head()

,FirstName,LastName,Email,Company,InvoiceId,InvoiceDate,BillingCountry,Total
0,Leonie,Köhler,leonekohler@surfeu.de,None,1,2009-01-01 00:00:00,Germany,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,None,2,2009-01-02 00:00:00,Norway,3.96
2,Daan,Peeters,daan_peeters@apple.be,None,3,2009-01-03 00:00:00,Belgium,5.94
3,Mark,Philips,mphilips12@shaw.ca,Telus,4,2009-01-06 00:00:00,Canada,8.91
4,John,Gordon,johngordon22@yahoo.com,None,5,2009-01-11 00:00:00,USA,13.86


**Pertanyaan:** Bagaimana jika kita ingin menghitung jumlah pelanggan yang bekerja di Apple Inc. dengan menggunakan domain `Email` sebagai pengganti indikator `Company`?

Silahkan lengkapi kode berikut:

```
applecust = pd.read_sql_query(
    '''
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices._____ = customers._____
    WHERE _____
    ''',
    conn
)
```

In [46]:
# your code here
applecust = pd.read_sql_query(
    '''
    SELECT firstname, lastname, email, company,
    invoiceid, invoicedate, billingcountry, total
    FROM invoices
    LEFT JOIN customers
    ON invoices.CustomerId = customers.CustomerId
    WHERE Email LIKE '%@apple%'
    ''',
    conn
)

applecust.shape

(49, 8)

Notes:

- Gunakan `LIKE` ketika kita ingin mencocokan sebuah pola, jangan pakai `=`
- Kasus ada customer yang menuliskan emailnya: mylovelyapple@gmail.com, maka agar customer tersebut tidak dideteksi sebagai karyawan Apple, gunakanlah polanya `%@apple%`

Berdasarkan query tersebut, berapa pelanggan yang bekerja di Apple Inc.?

- [ ] 412  
- [x] 49   
- [ ] 7  
- [ ] 14  

## Summary

### Knowledge Check 1

Kita telah mempelajari banyak statement pada SQL, silahkan mencocokan statement sesuai dengan kegunaannya:

**KEGUNAAN**

A. Memberikan nama lain pada tabel maupun kolom

B. Mengambil beberapa baris teratas dari tabel

C. Mengurutkan baris berdasarkan nilai pada kolom

D. Mengambil kolom dari sebuah tabel

E. Menggabungkan dua tabel menjadi satu tabel berdasarkan kolom penghubung

F. Filter baris

G. Membuat tabel agregasi

**STATEMENT**

1. `SELECT <nama_kolom> FROM <nama_tabel>`

2. `LIMIT <banyaknya_baris>`

3. `AS <nama_kolom_atau_baris>`

4. `<tabel_kiri> [LEFT|INNER] JOIN <tabel_kanan> ON <tabel>.key = <tabel>.key`

5. `GROUP BY <nama_kolom>`

6. `ORDER BY <nama_kolom> [ASC|DESC]`

7. `WHERE <kondisi>`

**JAWABAN**

1. D
2. B -> `.head()`
3. A
4. E -> `.merge()`
5. G -> `.groupby()`
6. C -> `.sort_values()`
7. F -> conditional subsetting: `data[condition]`

### Knowledge Check 2

Susunlah 8 baris SQL statement berikut menjadi sebuah kerangka urutan syntax yang benar:

1. `GROUP BY <nama_kolom>`

2. `LIMIT <banyaknya_baris>`

3. `SELECT <nama_kolom> AS ...`

4. `[LEFT|INNER] JOIN <tabel_kanan> AS ...`

5. `ORDER BY <nama_kolom> [ASC|DESC]`

6. `FROM <nama_tabel> AS ...`

7. `WHERE <kondisi>`
    
8. `ON <tabel>.key = <tabel>.key`

> **JAWABAN**: Urutan yang benar adalah 3-6-4-8-7-1-5-2

```
SELECT <nama_kolom> AS ...
FROM <nama_tabel> AS ...
[LEFT|INNER] JOIN <tabel_kanan> AS ...
ON <tabel>.key = <tabel>.key
WHERE <kondisi>
GROUP BY <nama_kolom>
ORDER BY <nama_kolom> [ASC|DESC]
LIMIT <banyaknya_baris>
```

**END OF DAY 2**

---

**START OF DAY 3**

### Dive Deeper

Toko musik digital ini ingin memberikan satu penghargaan ke `employees` yang telah bekerja keras melakukan penjualan di `BillingCountry` Amerika (yaitu **USA dan Canada**). Apabila penghargaan tersebut diberikan kepada `employees` berdasarkan **jumlah `Total`** penjualan, siapakah yang berhak mendapatkannya? Tampilkan **nama lengkap (`FirstName` dan `LastName`) beserta total penjualannya**.

In [47]:
# concate FirstName dan LastName
pd.read_sql_query(
    '''
    SELECT FirstName, LastName,
    FirstName || ' ' || LastName AS FullName
    FROM employees
    ''', conn
)

,FirstName,LastName,FullName
0,Andrew,Adams,Andrew Adams
1,Nancy,Edwards,Nancy Edwards
2,Jane,Peacock,Jane Peacock
3,Margaret,Park,Margaret Park
4,Steve,Johnson,Steve Johnson
5,Michael,Mitchell,Michael Mitchell
6,Robert,King,Robert King
7,Laura,Callahan,Laura Callahan


In [ ]:
# your code here



# Under and Over Fetching

Di antara semua tools yang telah kita pelajari untuk menganalisis data, sekarang saatnya kita merenungkan yang mana yang lebih cocok untuk Anda. Untuk meninjau, mari kita mengingat kembali apa yang telah kita pelajari:

- Reading flat files (CSV file)
- Data cleansing and wrangling
- Exploratory data analysis tools
- Visual exploratory tools

Kira-kira SQL masuk ke bagian apa? Untuk menjawab hal tersebut, Anda perlu memahami arsitektur client-server.

![](assets/clientserver.png)

Sedikit berbeda dengan `pandas` yang semua operasinya dilakukan di komputer lokal Anda. Ketika Anda bekerja dengan SQL, kemungkinan besar Anda memiliki database relasional yang disimpan terpusat pada server dan dapat diakses oleh beberapa client.

Saat Anda melakukan query, sebenarnya Anda sedang menjalankan perintah untuk mengunduh data ke komputer lokal. Proses pengunduhan ini membutuhkan sumber daya dan perlu memanfaatkan alat secara efektif untuk meminimalkan biaya.

- Over fetching adalah kondisi dimana kita menarik data dari database lebih dari yang dibutuhkan, sehingga membutuhkan biaya dan waktu yang lebih.

- Under fetching adalah kondisi sebaliknya, dimana data yang ditarik kurang dari yang dibutuhkan, sehingga proses analisis tidak dapat dilakukan secara lengkap.

**Diskusi:**

Anda diminta untuk melakukan analisis terhadap semua penjualan (`invoices`) genre `Rock` pada tahun 2012. Pertimbangkan pertanyaan berikut:

- Apakah perlu bagi Anda untuk mengunduh semua tabel `tracks` ke komputer lokal?
- Apakah Anda melakukan filter baris terhadap `tracks` dengan genre `Rock` menggunakan SQL statement `WHERE` atau conditional subsetting `pandas`?
- Karena kita memerlukan informasi dari beberapa tabel, manakah cara yang lebih nyaman: melakukan query dengan `JOIN` atau melakukan `SELECT` secara terpisah dari database?

Cobalah membuat query yang paling optimum menurut Anda:

In [ ]:
# your code here


# (Optional) SQL Subquery

Dalam beberapa kasus tertentu, kita ingin melakukan filter baris berdasarkan syarat tertentu dimana nilai-nilai kondisi didapatkan dari hasil query lain. Apakah Anda masih ingat bagaimana kita mengambil semua pelanggan yang memiliki `invoices` dengan Total paling tinggi? (Bagian SQL Aggregation)

In [ ]:
# menampilkan top 5 customers dengan total pembelanjaan terbanyak
pd.read_sql_query(
    '''
    SELECT CustomerId, SUM(Total) AS TotalValue,
    COUNT(InvoiceId) AS Purchases
    FROM invoices
    GROUP BY CustomerId
    ORDER BY TotalValue DESC
    LIMIT 5
    ''',
    conn,
    index_col='CustomerId'
)

Misalnya dari tabel tersebut kita ingin menarik semua data `invoices` berdasarkan top 5 `customers`. Untuk melakukan hal tersebut, kita akan menggunakan subquery setelah statement `WHERE` menggunakan operator `IN`. Sebenarnya kita bisa saja menuliskan list `CustomerId` dalam kondisi secara **hard-code** seperti ini:

In [ ]:
# menampilkan invoices hanya untuk top 5 customers
top5cust_hardcode = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE CustomerId IN (6, 26, 57, 45, 46)
    ''',
    conn)

top5cust_hardcode

Namun cara hard-code seperti itu tidak disarankan. Bagaimana kalau database kita terus bertambah, sehingga bukan lagi `CustomerId` 6, 26, 57, 45, 46 yang merupakan top 5 customers? Maka dari itu, kita perlu menggunakan subquery agar **query kita lebih dinamis terhadap perubahan data**.

Untuk itu mari kita persiapkan **subquery** yang hanya mengembalikan list Top 5 `CustomerId` berdasarkan `Total` pembelian:

In [ ]:
# subquery
pd.read_sql_query(
    '''
    SELECT CustomerId
    FROM invoices
    GROUP BY CustomerId
    ORDER BY SUM(Total) DESC
    LIMIT 5
    ''',
    conn)

Subquery di atas kita gunakan untuk menggantikan query yang hard-code:

In [ ]:
top5cust_subquery = pd.read_sql_query(
    '''
    SELECT *
    FROM invoices
    WHERE CustomerId IN (
        SELECT CustomerId
        FROM invoices
        GROUP BY CustomerId
        ORDER BY SUM(Total) DESC
        LIMIT 5
    )
    ''',
    conn)

top5cust_subquery

Hasil `top5cust_hardcode` dengan `top5cust_subquery` akan sama persis:

In [ ]:
top5cust_hardcode.shape

In [ ]:
top5cust_subquery.shape